In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
from random import randint
from sklearn.calibration import LabelEncoder
from common import get_full_data

def heart_failure():
    df = pd.read_csv("dataset/heart_failure_clinical_records_dataset.csv")
    # get dependent and independent features
    X=df.iloc[:,:-1]
    y=df.iloc[:,-1]
    classes = y.unique()
    X,y=get_full_data(X,y)
    return X,y,[str(c) for c in classes]

def encode_categorical_columns(df):
    le = LabelEncoder()
    # Loop over all columns in the DataFrame
    for col in df.columns:
        # Check if the column is of type object (string)
        if df[col].dtype == 'object':
            # Use LabelEncoder to do the numeric transformation
            df[col] = le.fit_transform(df[col])
    return df

def disease():
    df = pd.read_csv("dataset/disease.csv")
    # get dependent and independent features
    symptoms = df.iloc[:,1:]
    all_symptoms = set()
    for col in symptoms.columns:
        unq = symptoms[col].unique()
        unq=[str.strip(v) for v in unq if isinstance(v,str)]
        all_symptoms.update(unq)
    
    all_symptoms=list(all_symptoms)
    symptoms_embedding = pd.DataFrame(
        np.zeros((len(df),len(all_symptoms))),
        columns=all_symptoms
    )
    
    uniques = symptoms.apply(lambda x: np.unique(np.array(x.dropna(),dtype=str)),axis=1)
    for id,u in enumerate(uniques):
        u=[str.strip(v) for v in u if isinstance(v,str)]
        symptoms_embedding.iloc[id][u]=1

    y = df.iloc[:,0]
    classes = y.unique()
    
    for index,cls in enumerate(classes):
        y[y==cls]=index
    
    return symptoms_embedding, y.astype(int), classes

def AIDS():
    df = pd.read_csv("dataset/AIDS_Classification.csv")
    X=df.iloc[:,:-1]
    y=df.iloc[:,-1]
    classes = y.unique()
    X,y=get_full_data(X,y)
    return X,y,[str(c) for c in classes]

def seeds():
    df = pd.read_csv("dataset/seeds.csv")
    y = df.iloc[:,-1]-1
    classes = y.apply(lambda x: str(x)).unique()
    X = df.iloc[:,:-1]
    return X,y, classes

def housing():
    df = pd.read_csv("dataset/housing.csv")
    y=df['SaleCondition']
    bad_classes_names=['Alloca','AdjLand','Normal','Partial']
    classes = set(y.apply(lambda x: str(x)).unique())-set(bad_classes_names)

    X=df.drop(columns=['SaleCondition','SalePrice'])
    bad_classes = np.any([y == cl for cl in bad_classes_names],axis=0)
    X=encode_categorical_columns(X)

    X=np.array(X)
    y=np.array(y)

    y=y[~bad_classes]
    X=X[~bad_classes]


    y_encoded = np.zeros_like(y,dtype=np.int32)
    for i,cls in enumerate(classes):
        y_encoded[y==cls]=i

    return X,y_encoded,classes

In [ ]:
from xgboost import XGBClassifier
X,y, y_classes = housing()

# for high-dimensional data use `gpu` for device if you have one
special_model = XGBClassifier(device='cpu')

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from common import XGB_search_params

params = XGB_search_params()
state = randint(0,1000)
search = RandomizedSearchCV(
    special_model,
    params,
    n_iter=200,
    cv=5,
    random_state=state,
    n_jobs=-1,
)

# search.fit(X,y)
# special_model=search.best_estimator_

In [ ]:
# do repeated stratified k-fold cross-validation with classification report
from sklearn.model_selection import RepeatedStratifiedKFold
from common import cross_val_classification_report

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=50)
report = cross_val_classification_report(
    model=special_model,
    X=np.array(X),
    y=y,
    cv=cv,
    target_names=y_classes
)
print(report)

In [ ]:
# New method
from common import find_outliers

X_numpy = np.array(X)
y_numpy = np.array(y)

outliers_mask, score = find_outliers(
    X_numpy,
    y_numpy,
    special_model,
    outliers_to_remove=0.3,
    iterations=5,
    gamma=0.9,
    evaluate_loss=metrics.mean_squared_error,
    cv=5,
    repeats=3,
    class_weight_scale_power=0.5,
    plot=True
)

X_clean = X_numpy[~outliers_mask]
y_clean = y_numpy[~outliers_mask]

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=50)
report = cross_val_classification_report(
    model=special_model,
    X=X_clean,
    y=y_clean,
    cv=cv,
    target_names=y_classes
)
print(report)


In [ ]:
from common import generate_colors_for_classification
from sklearn.preprocessing import StandardScaler, RobustScaler
from kernel_pca_search import KernelPCASearchCV, kernel_pca_scorer
from render import *

scaler = RobustScaler()

X_numpy[np.isnan(X_numpy)]=-1
X_ = scaler.fit_transform(X_numpy)

transform = KernelPCASearchCV(n_components=3,n_iter=-1,kernel='rbf')
x_transform = transform.fit_transform(X_)

colors = generate_colors_for_classification(y,seed=100)
data = np.concatenate([x_transform,colors],axis=1)
plot_3d_rgb(data,"Original data plot",["d1","d2","d3"],6,None)

colors[outliers_mask]=(0,0,0)
data = np.concatenate([x_transform,colors],axis=1)
plot_3d_rgb(data,"Cleaned data plot",["d1","d2","d3"],6,None)

print(kernel_pca_scorer(transform,X_))


Compare performance with some control model `SVC` with `rbf` kernel and standard scaler

Find optimal hyperparameters using `RandomSearchCV`